In [62]:
import requests
import json
import time
from typing import Optional, Dict, Any
import pandas as pd

In [73]:

class WebRenderTool:
    """网页显示工具测试类"""
    
    def __init__(self, base_url: str = "https://www.cisdi.net.cn/web-render"):
        self.base_url = base_url
        self.session = requests.Session()
        
    def health_check(self) -> bool:
        """健康检查"""
        try:
            response = self.session.get(f"{self.base_url}/health")
            if response.status_code == 200:
                return True
            else:
                return False
        except Exception as e:
            return False
    
    def upload_webpage(self, html_content: str, title: Optional[str] = None) -> Optional[Dict[str, Any]]:
        """上传网页"""
        try:
            data = {
                "html_content": html_content
            }
            if title:
                data["title"] = title
                
            response = self.session.post(
                f"{self.base_url}/api/pages",
                json=data,
                headers={"Content-Type": "application/json"}
            )
            
            if response.status_code == 201:
                result = response.json()
                return { 
                    "page_id"  : result['page_id'] ,  
                    "base_url" : self.base_url + result['url']
                }
            else:
                return None
        except Exception as e:
            return None
    
    def get_page_info(self, page_id: str) -> Optional[Dict[str, Any]]:
        """获取页面信息"""
        try:
            response = self.session.get(f"{self.base_url}/api/pages/{page_id}")
            
            if response.status_code == 200:
                result = response.json()
                return result
            else:
                return None
        except Exception as e:
            return None
    
    def list_all_pages(self, skip: int = 0, limit: int = 100) -> Optional[list]:
        """获取所有页面列表"""
        try:
            params = {"skip": skip, "limit": limit}
            response = self.session.get(f"{self.base_url}/api/pages", params=params)
            
            if response.status_code == 200:
                result = response.json()
                return pd.DataFrame(result)
            else:
                return None
        except Exception as e:
            return None
    
    def view_page(self, page_id: str) -> bool:
        """访问页面（获取HTML内容）"""
        try:
            response = self.session.get(f"{self.base_url}/view/{page_id}")
            
            if response.status_code == 200:
                return response.text
            else:
                return None
        except Exception as e:
            return None
    
    def delete_page(self, page_id: str) -> bool:
        """删除页面"""
        try:
            response = self.session.delete(f"{self.base_url}/api/pages/{page_id}")
            if response.status_code == 200:
                result = response.json()
                return True
            else:
                return False
        except Exception as e:
            return False
    
    
    

In [83]:
# 初始化测试器
webRenderTool = WebRenderTool()
# 1. 健康检查
webRenderTool.health_check() # True
# 4. 上传测试网页
test_html = """
<!DOCTYPE html>
<html>
<head>
    <title>测试页面</title>
    <style>
        body { font-family: Arial, sans-serif; margin: 40px; }
        .header { color: #333; text-align: center; }
        .content { background: #f5f5f5; padding: 20px; border-radius: 8px; }
    </style>
</head>
<body>
    <h1 class="header">这是一个测试页面</h1>
    <div class="content">
        <p>这是通过API上传的测试网页内容。</p>
        <p>当前时间: <span id="time"></span></p>
        <script>
            document.getElementById('time').textContent = new Date().toLocaleString();
        </script>
    </div>
</body>
</html>
"""

upload_result = webRenderTool.upload_webpage(test_html, "API测试页面")
# {'page_id': '9136e779','base_url': 'https://www.cisdi.net.cn/web-render/view/9136e779'}
# 5. 获取页面信息
print("\n📋 5. 获取页面信息")
webRenderTool.get_page_info(page_id)
# {'id': 1,'page_id': '9136e779','title': 'API测试页面', 'html_content':
# 6. 访问页面
print("\n📋 6. 访问页面")
webRenderTool.view_page(page_id)
# 8. 获取所有页面列表
print("\n📋 8. 获取所有页面列表")
pageDf = webRenderTool.list_all_pages() 
webRenderTool.delete_page('e073d75b') # False


📋 5. 获取页面信息

📋 6. 访问页面

📋 8. 获取所有页面列表


False